In [ ]:
# Databricks Notebook
# ===================================
# 02_Bronze_Layer
# ===================================

from pyspark.sql.types import *
from pyspark.sql.functions import *

print("="*60)
print("ブロンズレイヤー構築開始")
print("="*60)

# ===================================
# 1. 商品マスタ（JSON）の取り込み
# ===================================

print("\n[1/3] 商品マスタデータ取り込み中...")

# JSONスキーマ定義（オプション：パフォーマンス向上のため）
items_schema = StructType([
    StructField("item_id", StringType(), True),
    StructField("item_name", StringType(), True),
    StructField("category_l1", StringType(), True),
    StructField("category_l2", StringType(), True),
    StructField("category_l3", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("price", IntegerType(), True),
    StructField("cost", IntegerType(), True),
    StructField("color", StringType(), True),
    StructField("size", StringType(), True),
    StructField("season", StringType(), True),
    StructField("launch_date", StringType(), True),
    StructField("is_active", BooleanType(), True)
])

# JSONファイル読み込み
items_raw_df = spark.read \
    .schema(items_schema) \
    .json("/FileStore/sample_data/items/*.json")

# メタデータ追加
items_bronze_df = items_raw_df \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .withColumn("source_system", lit("product_master_db")) \
    .withColumn("file_name", input_file_name())

# Deltaテーブルとして保存（ブロンズレイヤー）
items_bronze_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{BRONZE_PATH}/items")

# テーブル登録
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze_items
    USING DELTA
    LOCATION '{BRONZE_PATH}/items'
""")

print(f"✓ 商品マスタ取り込み完了: {items_bronze_df.count():,}件")

# ===================================
# 2. 広告データ（CSV）の取り込み
# ===================================

print("\n[2/3] 広告データ取り込み中...")

# CSVスキーマ定義
ads_schema = StructType([
    StructField("campaign_id", StringType(), True),
    StructField("campaign_name", StringType(), True),
    StructField("ad_id", StringType(), True),
    StructField("ad_platform", StringType(), True),
    StructField("ad_format", StringType(), True),
    StructField("target_url", StringType(), True),
    StructField("impressions", IntegerType(), True),
    StructField("clicks", IntegerType(), True),
    StructField("cost", DoubleType(), True),
    StructField("date", StringType(), True),
    StructField("utm_source", StringType(), True),
    StructField("utm_medium", StringType(), True),
    StructField("utm_campaign", StringType(), True),
    StructField("target_category", StringType(), True)
])

# CSVファイル読み込み
ads_raw_df = spark.read \
    .schema(ads_schema) \
    .option("header", "true") \
    .csv("/FileStore/sample_data/digital_ads/*-ads.csv")

# メタデータ追加
ads_bronze_df = ads_raw_df \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .withColumn("source_system", lit("google_ads_api")) \
    .withColumn("file_name", input_file_name())

# Deltaテーブルとして保存
ads_bronze_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{BRONZE_PATH}/digital_ads")

# テーブル登録
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze_digital_ads
    USING DELTA
    LOCATION '{BRONZE_PATH}/digital_ads'
""")

print(f"✓ 広告データ取り込み完了: {ads_bronze_df.count():,}件")

# ===================================
# 3. トランザクションデータ（CSV）の取り込み
# ===================================

print("\n[3/3] トランザクションデータ取り込み中...")

# CSVスキーマ定義
transactions_schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("transaction_timestamp", StringType(), True),
    StructField("item_id", StringType(), True),
    StructField("user_email", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("price", IntegerType(), True),
    StructField("referrer_url", StringType(), True),
    StructField("landing_page_url", StringType(), True),
    StructField("utm_source", StringType(), True),
    StructField("utm_medium", StringType(), True),
    StructField("utm_campaign", StringType(), True),
    StructField("device_type", StringType(), True),
    StructField("session_id", StringType(), True),
    StructField("conversion_time_minutes", IntegerType(), True)
])

# CSVファイル読み込み
transactions_raw_df = spark.read \
    .schema(transactions_schema) \
    .option("header", "true") \
    .csv("/FileStore/sample_data/transactions/*-transactions.csv")

# メタデータ追加
transactions_bronze_df = transactions_raw_df \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .withColumn("source_system", lit("ec_site_db")) \
    .withColumn("file_name", input_file_name())

# Deltaテーブルとして保存
transactions_bronze_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{BRONZE_PATH}/transactions")

# テーブル登録
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze_transactions
    USING DELTA
    LOCATION '{BRONZE_PATH}/transactions'
""")

print(f"✓ トランザクションデータ取り込み完了: {transactions_bronze_df.count():,}件")

# ===================================
# ブロンズレイヤー完成確認
# ===================================

print("\n" + "="*60)
print("ブロンズレイヤー構築完了")
print("="*60)

# テーブル一覧表示
print("\n【作成されたテーブル】")
display(spark.sql("SHOW TABLES IN ad_analytics"))

# サンプルデータ表示
print("\n【ブロンズ: 商品マスタ（先頭5件）】")
display(spark.table("bronze_items").limit(5))

print("\n【ブロンズ: 広告データ（先頭5件）】")
display(spark.table("bronze_digital_ads").limit(5))

print("\n【ブロンズ: トランザクション（先頭5件）】")
display(spark.table("bronze_transactions").limit(5))